In [21]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
from datetime import datetime, timedelta



In [28]:
sp_all = pd.read_csv('../data/temp/sp500/sp500.csv', low_memory=False,parse_dates=['Date'])
cpi = pd.read_csv("../data/temp/cpi/cpi_alpha-1.csv")
inflation = pd.read_csv("../data/temp/inflation/InflationRates.csv")
intRate = pd.read_csv("../data/int_rates_cleaned_final.csv")
nfpayroll = pd.read_csv("../data/nfpayroll_cleaned-final.csv")
unemployment = pd.read_csv("../data/temp/unemployment/unemployment_bls.csv")

,DateCPI,CPI
0,1/1/1913,9.800
1,2/1/1913,9.800
2,3/1/1913,9.800
3,4/1/1913,9.800
4,5/1/1913,9.700
...,...,...
1319,12/1/2022,296.797
1320,1/1/2023,299.170
1321,2/1/2023,300.840
1322,3/1/2023,301.836


# 1. S&P500 Cleanup

In [23]:
sp_all['Date'] = pd.to_datetime(sp_all['Date'], utc=True).dt.date
sp_all['Date'] = sp_all['Date'].astype('datetime64[ns]')
sp = sp_all
#Drop columns which are not needed, 'Volume', 'Dividends', 'Stock Splits'
#Sort dataframe
sp = sp.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1).sort_values(by='Date',ascending=True) 
#Round to 2 decimal places
sp = sp.round(2)
sp

,Date,Open,High,Low,Close
0,1927-12-30,17.66,17.66,17.66,17.66
1,1928-01-03,17.76,17.76,17.76,17.76
2,1928-01-04,17.72,17.72,17.72,17.72
3,1928-01-05,17.55,17.55,17.55,17.55
4,1928-01-06,17.66,17.66,17.66,17.66
...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77


In [ ]:
sp.to_csv('../data/temp/sp500/sp500_cleaned-final.csv', index=False)  

In [24]:
sp = pd.read_csv("../data/temp/sp500/sp500_cleaned-final.csv")
sp

,Date,Open,High,Low,Close
0,1927-12-30,17.66,17.66,17.66,17.66
1,1928-01-03,17.76,17.76,17.76,17.76
2,1928-01-04,17.72,17.72,17.72,17.72
3,1928-01-05,17.55,17.55,17.55,17.55
4,1928-01-06,17.66,17.66,17.66,17.66
...,...,...,...,...,...
23953,2023-05-12,4138.54,4143.74,4099.12,4124.08
23954,2023-05-15,4126.65,4141.25,4110.27,4136.28
23955,2023-05-16,4127.95,4135.54,4109.86,4109.90
23956,2023-05-17,4122.85,4164.67,4113.62,4158.77


### 1.1 Get just the close column for processing other economic indicators This df will be used for joining the indicatorts

In [26]:
sp['Date'] = pd.to_datetime(sp['Date'], utc=True).dt.date
sp['Date'] = sp['Date'].astype('datetime64[ns]')

#Drop columns which are not needed, 'Volume', 'Dividends', 'Stock Splits'
#Sort dataframe
sp = sp.drop(['Open','High','Low','Close'], axis=1).sort_values(by='Date',ascending=True) 
sp = sp.sort_values(by=['Date'], ascending=True)
sp

,Date
0,1927-12-30
1,1928-01-03
2,1928-01-04
3,1928-01-05
4,1928-01-06
...,...
23953,2023-05-12
23954,2023-05-15
23955,2023-05-16
23956,2023-05-17


# CPI- Cleanup

In [29]:
release = pd.read_csv("../data/temp/cpi/TempR-1.csv")
release

,Date
0,3/24/1949
1,4/22/1949
2,5/23/1949
3,6/24/1949
4,7/22/1949
...,...
886,1/12/2023
887,2/10/2023
888,3/14/2023
889,4/12/2023


In [30]:
# Convert 'Date' columns to datetime
cpi['Date'] = pd.to_datetime(cpi['DateCPI'])
release['Date'] = pd.to_datetime(release['Date'])

# Extract month and year from 'Date' columns
cpi['Year'] = cpi['Date'].dt.year
cpi['Month'] = cpi['Date'].dt.month
release['Year'] = release['Date'].dt.year
release['Month'] = release['Date'].dt.month

# Merge DataFrames on matching Year and Month
cpi_merged_df = pd.merge(cpi, release, on=['Year', 'Month'])

cpi_merged_df

,DateCPI,CPI,Date_x,Year,Month,Date_y
0,3/1/1949,23.800,1949-03-01,1949,3,1949-03-24
1,4/1/1949,23.900,1949-04-01,1949,4,1949-04-22
2,5/1/1949,23.800,1949-05-01,1949,5,1949-05-23
3,6/1/1949,23.900,1949-06-01,1949,6,1949-06-24
4,7/1/1949,23.700,1949-07-01,1949,7,1949-07-22
...,...,...,...,...,...,...
885,12/1/2022,296.797,2022-12-01,2022,12,2022-12-13
886,1/1/2023,299.170,2023-01-01,2023,1,2023-01-12
887,2/1/2023,300.840,2023-02-01,2023,2,2023-02-10
888,3/1/2023,301.836,2023-03-01,2023,3,2023-03-14


In [31]:
cpi_merged_df["DateCPI"] = cpi_merged_df["Date_y"]  
cpi_merged_df = cpi_merged_df.drop(columns =['Date_x', 'Date_y', 'Year', 'Month'])
cpi_merged_df = cpi_merged_df.rename(columns={"DateCPI": "Date"})
cpi_merged_df

,Date,CPI
0,1949-03-24,23.800
1,1949-04-22,23.900
2,1949-05-23,23.800
3,1949-06-24,23.900
4,1949-07-22,23.700
...,...,...
885,2022-12-13,296.797
886,2023-01-12,299.170
887,2023-02-10,300.840
888,2023-03-14,301.836


In [ ]:
#cpi_merged_df.to_csv('../data/temp/cpi_merged_df-2.csv', index=False)  

In [34]:
release = pd.read_csv("../data/temp/cpi/cpi_merged_df-2.csv")
release

,Date,CPI
0,1949-03-24,23.800
1,1949-04-22,23.900
2,1949-05-23,23.800
3,1949-06-24,23.900
4,1949-07-22,23.700
...,...,...
885,2022-12-13,296.797
886,2023-01-12,299.170
887,2023-02-10,300.840
888,2023-03-14,301.836


In [35]:
release.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    890 non-null    object 
 1   CPI     890 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.0+ KB


In [36]:
release['Date'] = release['Date'].astype('datetime64[ns]')

In [37]:
#release['Date'] = release['Date'].astype('datetime64[ns]')
#r1 = release.groupby(release.Date.dt.year)['CPI'].count() 
r1 = release.groupby(release.Date.dt.year)['CPI'].count().reset_index(name='count')
r1

,Date,count
0,1949,10
1,1950,12
2,1951,12
3,1952,12
4,1953,12
...,...,...
70,2019,12
71,2020,12
72,2021,12
73,2022,12


In [38]:
release

,Date,CPI
0,1949-03-24,23.800
1,1949-04-22,23.900
2,1949-05-23,23.800
3,1949-06-24,23.900
4,1949-07-22,23.700
...,...,...
885,2022-12-13,296.797
886,2023-01-12,299.170
887,2023-02-10,300.840
888,2023-03-14,301.836


In [39]:
#Since cpi is released only once a month, we don't have data for every day
#we need data for every day when we join the cpi dataframe with sp500 df.
#Approach we will have is forward fill the same value in cpi by create dates for every month
#Get only Dates from SP df, this will be used to join cpi for forward fill
#Left join sp with cpi release
spf= sp.merge(release, on='Date', how='left').sort_values(by=['Date'], ascending=True)
spf

,Date,CPI
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,NaN
23954,2023-05-15,NaN
23955,2023-05-16,NaN
23956,2023-05-17,NaN


In [40]:
spf[spf['Date'] > '2021-03-28'].head(30)

,Date,CPI
23418,2021-03-29,NaN
23419,2021-03-30,NaN
23420,2021-03-31,NaN
23421,2021-04-01,NaN
23422,2021-04-05,NaN
23423,2021-04-06,NaN
23424,2021-04-07,NaN
23425,2021-04-08,NaN
23426,2021-04-09,NaN
23427,2021-04-12,NaN


In [41]:
spf['CPI'] = spf['CPI'].fillna(method='ffill')
spf

,Date,CPI
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,303.363
23954,2023-05-15,303.363
23955,2023-05-16,303.363
23956,2023-05-17,303.363


In [ ]:
#spf.to_csv('../data/temp/cpi_alpha_cleaned-final.csv', index=False)  

In [43]:
cpi = pd.read_csv("../data/temp/cpi/cpi_alpha_cleaned-final.csv")
cpi

,Date,CPI
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,303.363
23954,2023-05-15,303.363
23955,2023-05-16,303.363
23956,2023-05-17,303.363


# Inflation Rates - Cleanup

In [45]:
inflation

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Ave
0,1914,2.0,1.0,1.0,0.0,2.1,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0
1,1915,1.0,1.0,0.0,2.0,2.0,2.0,1.0,-1.0,-1.0,1.0,1.0,2.0,1.0
2,1916,3.0,4.0,6.1,6.0,5.9,6.9,6.9,7.9,9.9,10.8,11.7,12.6,7.9
3,1917,12.5,15.4,14.3,18.9,19.6,20.4,18.5,19.3,19.8,19.5,17.4,18.1,17.4
4,1918,19.7,17.5,16.7,12.7,13.3,13.1,18.0,18.5,18.0,18.5,20.7,20.4,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,2019,1.6,1.5,1.9,2.0,1.8,1.6,1.8,1.7,1.7,1.8,2.1,2.3,1.8
106,2020,2.5,2.3,1.5,0.3,0.1,0.6,1.0,1.3,1.4,1.2,1.2,1.4,1.2
107,2021,1.4,1.7,2.6,4.2,5.0,5.4,5.4,5.3,5.4,6.2,6.8,7.0,4.7
108,2022,7.5,7.9,8.5,8.3,8.6,9.1,8.5,8.3,8.2,7.7,7.1,6.5,8.0


In [46]:

#Drop Average column
inflation = inflation.drop(['Ave'], axis=1)
inflation = inflation.melt(id_vars='Year', var_name='Month', value_name='Value')
inflation = inflation.dropna()

# # Map month abbreviations to numerical representation
inflation['Month'] = inflation['Month'].apply(lambda x: datetime.strptime(x, '%b').month)

# # Concatenate Year and Month columns
inflation['Year'] = inflation['Year'].astype(str) + '-' + inflation['Month'].astype(str)
inflation= inflation.drop(['Month'], axis=1)
inflation['Year'] = pd.to_datetime(inflation['Year'], utc=True).dt.date
inflation = inflation.sort_values(by=['Year'], ascending=True)

inflation = inflation.rename(columns={"Year": "Date"})
inflation['Date'] = pd.to_datetime(inflation['Date'], utc=True).dt.date
inflation['Date'] = inflation['Date'].astype('datetime64[ns]')
inflation = inflation.reset_index(drop=True)
inflation



,Date,Value
0,1914-01-01,2.0
1,1914-02-01,1.0
2,1914-03-01,1.0
3,1914-04-01,0.0
4,1914-05-01,2.1
...,...,...
1307,2022-12-01,6.5
1308,2023-01-01,6.4
1309,2023-02-01,6.0
1310,2023-03-01,5.0


In [48]:
#Left join sp with inflation
spi= sp.merge(inflation, on='Date', how='left').sort_values(by=['Date'], ascending=True)
spi

,Date,Value
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,NaN
23954,2023-05-15,NaN
23955,2023-05-16,NaN
23956,2023-05-17,NaN


In [49]:
nan_count = spi.isna().sum() 
nan_count

Date         0
Value    23227
dtype: int64

In [50]:
spi['Value'] = spi['Value'].fillna(method='ffill')
spi

,Date,Value
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,5.0
23954,2023-05-15,5.0
23955,2023-05-16,5.0
23956,2023-05-17,5.0


In [51]:
spi[spi['Date'] > '2021-01-15'].head(30)

,Date,Value
23370,2021-01-19,1.4
23371,2021-01-20,1.4
23372,2021-01-21,1.4
23373,2021-01-22,1.4
23374,2021-01-25,1.4
23375,2021-01-26,1.4
23376,2021-01-27,1.4
23377,2021-01-28,1.4
23378,2021-01-29,1.4
23379,2021-02-01,1.7


In [52]:
spi.to_csv('../data/temp/inflation/inflation_cleaned-final.csv', index=False)  

In [53]:
dfi = pd.read_csv("../data/temp/inflation/inflation_cleaned-final.csv")
dfi

,Date,Value
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,5.0
23954,2023-05-15,5.0
23955,2023-05-16,5.0
23956,2023-05-17,5.0


# Interest Rates - Cleanup

In [54]:
intRate = pd.read_csv("../data/int_rates_cleaned_final.csv")

In [55]:
intRate

,Date,IntRate
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,5.08
23954,2023-05-15,5.08
23955,2023-05-16,5.08
23956,2023-05-17,5.08


In [56]:
intRate['Date'] = pd.to_datetime(intRate['Date'], utc=True).dt.date
intRate['Date'] = intRate['Date'].astype('datetime64[ns]')
intRate = intRate.sort_values(by=['Date'], ascending=True)
intRate

,Date,IntRate
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,5.08
23954,2023-05-15,5.08
23955,2023-05-16,5.08
23956,2023-05-17,5.08


In [57]:
nan_count = intRate.isna().sum() 
nan_count

Date          0
IntRate    6620
dtype: int64

In [58]:
intRate['Date'] = pd.to_datetime(intRate['Date'], utc=True).dt.date
intRate['Date'] = intRate['Date'].astype('datetime64[ns]')
intRate

,Date,IntRate
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,5.08
23954,2023-05-15,5.08
23955,2023-05-16,5.08
23956,2023-05-17,5.08


In [59]:
#Left join sp with intRate
spint= sp.merge(intRate, on='Date', how='left').sort_values(by=['Date'], ascending=True)
spint

,Date,IntRate
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,5.08
23954,2023-05-15,5.08
23955,2023-05-16,5.08
23956,2023-05-17,5.08


In [60]:
spint.to_csv('../data/temp/int_rates/int_rates_cleaned_final.csv', index=False)  

In [61]:
intr = pd.read_csv("../data/temp/int_rates/int_rates_cleaned_final.csv")
intr

,Date,IntRate
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,5.08
23954,2023-05-15,5.08
23955,2023-05-16,5.08
23956,2023-05-17,5.08


# Non Farm Payroll - Cleanup

In [65]:
nfpayroll = pd.read_csv("../data/nfpayroll_cleaned-final.csv")
nfpayroll

,Date,nfpayroll
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,155420.0
23954,2023-05-15,155420.0
23955,2023-05-16,155420.0
23956,2023-05-17,155420.0


In [66]:
nan_count = nfpayroll.isna().sum() 
nan_count

Date            0
nfpayroll    2768
dtype: int64

In [67]:
# Rename the columns
nfpayroll = nfpayroll.rename(columns={"DATE": "Date", "PAYEMS" :"nfpayroll"})
#Covert Date column from  object to DateTime
nfpayroll['Date'] = pd.to_datetime(nfpayroll['Date'], utc=True).dt.date

nfpayroll= nfpayroll.sort_values(by=['Date'], ascending=True)
nfpayroll

,Date,nfpayroll
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,155420.0
23954,2023-05-15,155420.0
23955,2023-05-16,155420.0
23956,2023-05-17,155420.0


In [68]:
nfpayroll

,Date,nfpayroll
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,155420.0
23954,2023-05-15,155420.0
23955,2023-05-16,155420.0
23956,2023-05-17,155420.0


In [69]:
nfpayroll['Date'] = pd.to_datetime(nfpayroll['Date'], utc=True).dt.date
nfpayroll['Date'] = nfpayroll['Date'].astype('datetime64[ns]')

In [70]:
#Left join sp with nfpayroll

spn= sp.merge(nfpayroll, on='Date', how='left').sort_values(by=['Date'], ascending=True)
spn

,Date,nfpayroll
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,155420.0
23954,2023-05-15,155420.0
23955,2023-05-16,155420.0
23956,2023-05-17,155420.0


In [71]:
spn[spn['Date'] > '2021-01-15'].head(30)

,Date,nfpayroll
23370,2021-01-19,142475.0
23371,2021-01-20,142475.0
23372,2021-01-21,142475.0
23373,2021-01-22,142475.0
23374,2021-01-25,142475.0
23375,2021-01-26,142475.0
23376,2021-01-27,142475.0
23377,2021-01-28,142475.0
23378,2021-01-29,142475.0
23379,2021-02-01,143544.0


In [72]:
spn['nfpayroll'] = spn['nfpayroll'].fillna(method='ffill')
spn

,Date,nfpayroll
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,155420.0
23954,2023-05-15,155420.0
23955,2023-05-16,155420.0
23956,2023-05-17,155420.0


In [73]:
spn[spn['Date'] > '2021-01-15'].head(30)

,Date,nfpayroll
23370,2021-01-19,142475.0
23371,2021-01-20,142475.0
23372,2021-01-21,142475.0
23373,2021-01-22,142475.0
23374,2021-01-25,142475.0
23375,2021-01-26,142475.0
23376,2021-01-27,142475.0
23377,2021-01-28,142475.0
23378,2021-01-29,142475.0
23379,2021-02-01,143544.0


In [74]:
spn.to_csv('../data/temp/nfpayroll/nfpayroll_cleaned-final.csv', index=False)  

In [75]:
nfp = pd.read_csv("../data/temp/nfpayroll/nfpayroll_cleaned-final.csv")
nfp

,Date,nfpayroll
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,155420.0
23954,2023-05-15,155420.0
23955,2023-05-16,155420.0
23956,2023-05-17,155420.0


# Enemployment Clean-up

In [77]:

unemployment

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1948,3.4,3.8,4.0,3.9,3.5,3.6,3.6,3.9,3.8,3.7,3.8,4.0
1,1949,4.3,4.7,5.0,5.3,6.1,6.2,6.7,6.8,6.6,7.9,6.4,6.6
2,1950,6.5,6.4,6.3,5.8,5.5,5.4,5.0,4.5,4.4,4.2,4.2,4.3
3,1951,3.7,3.4,3.4,3.1,3.0,3.2,3.1,3.1,3.3,3.5,3.5,3.1
4,1952,3.2,3.1,2.9,2.9,3.0,3.0,3.2,3.4,3.1,3.0,2.8,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2019,4.0,3.8,3.8,3.6,3.7,3.6,3.7,3.7,3.5,3.6,3.6,3.6
72,2020,3.5,3.5,4.4,14.7,13.2,11.0,10.2,8.4,7.9,6.9,6.7,6.7
73,2021,6.3,6.2,6.1,6.1,5.8,5.9,5.4,5.2,4.8,4.5,4.2,3.9
74,2022,4.0,3.8,3.6,3.6,3.6,3.6,3.5,3.7,3.5,3.7,3.6,3.5


In [78]:
unemployment = unemployment.melt(id_vars='Year', var_name='Month', value_name='Value')
unemployment = unemployment.dropna()

#Map month abbreviations to numerical representation
unemployment['Month'] = unemployment['Month'].apply(lambda x: datetime.strptime(x, '%b').month)

#Concatenate Year and Month columns
unemployment['Year'] = unemployment['Year'].astype(str) + '-' + unemployment['Month'].astype(str)
unemployment= unemployment.drop(['Month'], axis=1)
unemployment = unemployment.sort_values(by=['Year'], ascending=True)
unemployment = unemployment.rename(columns={"Year": "Date"})
unemployment['Date'] = pd.to_datetime(unemployment['Date'], utc=True).dt.date
unemployment= unemployment.sort_values(by=['Date'], ascending=True)
unemployment = unemployment.reset_index(drop=True)
unemployment

,Date,Value
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0
3,1948-04-01,3.9
4,1948-05-01,3.5
...,...,...
899,2022-12-01,3.5
900,2023-01-01,3.4
901,2023-02-01,3.6
902,2023-03-01,3.5


In [79]:
unemployment['Date'] = pd.to_datetime(unemployment['Date'], utc=True).dt.date
unemployment['Date'] = unemployment['Date'].astype('datetime64[ns]')
unemployment.sort_values(by=['Date'], ascending=True)
unemployment

,Date,Value
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0
3,1948-04-01,3.9
4,1948-05-01,3.5
...,...,...
899,2022-12-01,3.5
900,2023-01-01,3.4
901,2023-02-01,3.6
902,2023-03-01,3.5


In [80]:
unemployment['Date'] = pd.to_datetime(unemployment['Date'], utc=True).dt.date
unemployment['Date'] = unemployment['Date'].astype('datetime64[ns]')
u1 = unemployment.groupby(unemployment.Date.dt.year)['Value'].count().reset_index(name='count', drop = True)
#u1[u1['count'] > 12]
u1

0     12
1     12
2     12
3     12
4     12
      ..
71    12
72    12
73    12
74    12
75     4
Name: Value, Length: 76, dtype: int64

In [83]:
nan_count = u1.isna().sum() 
nan_count

0

In [84]:
spu['Value'] = spu['Value'].fillna(method='ffill')
spu

,Date,Value
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,3.5
23954,2023-05-15,3.5
23955,2023-05-16,3.5
23956,2023-05-17,3.5


In [85]:
spu[spu['Date'] > '2021-01-15'].head(30)

,Date,Value
23370,2021-01-19,6.7
23371,2021-01-20,6.7
23372,2021-01-21,6.7
23373,2021-01-22,6.7
23374,2021-01-25,6.7
23375,2021-01-26,6.7
23376,2021-01-27,6.7
23377,2021-01-28,6.7
23378,2021-01-29,6.7
23379,2021-02-01,6.2


In [86]:
spu.to_csv('../data/temp/unemployment/unemployment-cleaned-final.csv', index=False)  

In [87]:
unemp = pd.read_csv("../data/temp/unemployment/unemployment-cleaned-final.csv")
unemp

,Date,Value
0,1927-12-30,NaN
1,1928-01-03,NaN
2,1928-01-04,NaN
3,1928-01-05,NaN
4,1928-01-06,NaN
...,...,...
23953,2023-05-12,3.5
23954,2023-05-15,3.5
23955,2023-05-16,3.5
23956,2023-05-17,3.5
